In [25]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf
import numpy as np
from keras.layers import *
from keras.models import Model
from keras.utils import plot_model
import keras 

optimizers = keras.optimizers

In [15]:

class MyFlatten(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(MyFlatten, self).__init__(**kwargs)

    def compute_mask(self, inputs, mask=None):
        if mask==None:
            return mask
        return K.batch_flatten(mask)

    def call(self, inputs, mask=None):
        return K.batch_flatten(inputs)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], np.prod(input_shape[1:]))
    



class MyMeanPool(Layer):
    def __init__(self, axis, **kwargs):
        self.supports_masking = True
        self.axis = axis
        super(MyMeanPool, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        # need not to pass the mask to next layers
        return None

    def call(self, x, mask=None):
        if mask is not None:
            if K.ndim(x)!=K.ndim(mask):
                mask = K.repeat(mask, x.shape[-1])
                mask = tf.transpose(mask, [0,2,1])
            mask = K.cast(mask, K.floatx())
            x = x * mask
            return K.sum(x, axis=self.axis) / K.sum(mask, axis=self.axis)
        else:
            return K.mean(x, axis=self.axis)

    def compute_output_shape(self, input_shape):
        output_shape = []
        for i in range(len(input_shape)):
            if i!=self.axis:
                output_shape.append(input_shape[i])
        return tuple(output_shape)
    

class MySumLayer(Layer):
    def __init__(self, axis, **kwargs):
        self.supports_masking = True
        self.axis = axis
        super(MySumLayer, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        
        if mask is not None:
            # mask (batch, time)
            mask = K.cast(mask, K.floatx())
            if K.ndim(x)!=K.ndim(mask):
                mask = K.repeat(mask, x.shape[-1])
                mask = tf.transpose(mask, [0,2,1])
            x = x * mask
            if K.ndim(x)==2:
                x = K.expand_dims(x)
            return K.sum(x, axis=self.axis)
        else:
            if K.ndim(x)==2:
                x = K.expand_dims(x)
            return K.sum(x, axis=self.axis)

    def compute_output_shape(self, input_shape):
        output_shape = []
        for i in range(len(input_shape)):
            if i!=self.axis:
                output_shape.append(input_shape[i])
        if len(output_shape)==1:
            output_shape.append(1)
        return tuple(output_shape)

In [3]:
# numeric fields
USER_DIM=20
ACCOUNT_DIM=40

user_inputs = Input(shape=[USER_DIM], name="user") # None*USER_DIM
account_inputs = Input(shape=[ACCOUNT_DIM], name="account") # None*ACCOUNT_DIM

In [5]:
user_inputs, account_inputs

(<tf.Tensor 'user:0' shape=(?, 20) dtype=float32>,
 <tf.Tensor 'account:0' shape=(?, 40) dtype=float32>)

In [35]:
user_inputs_ = np.random.randint(0,2, (5120, USER_DIM))
account_inputs_ = np.random.randint(0,40, (5120, ACCOUNT_DIM))
label_ = np.random.randint(0,2, (5120, 1))

In [36]:
user_inputs_.shape, account_inputs_.shape, label_.shape

((5120, 20), (5120, 40), (5120, 1))

# y_first_order

In [11]:
# First Order Embeddings 
# dense_user = Dense(20)(user_inputs) # None*20
# dense_account = Dense(40)(account_inputs) # None*40
# dense_user, dense_account

emb_user = Reshape([USER_DIM])(Embedding(USER_DIM, 1)(user_inputs)) # None*1, 
emb_account = Reshape([ACCOUNT_DIM])(Embedding(ACCOUNT_DIM, 1)(account_inputs)) # None*1
print(emb_user, emb_account)

y_first_order = Concatenate(axis=1)([emb_user, emb_account])
print(y_first_order)

Instructions for updating:
Colocations handled automatically by placer.
Tensor("reshape_1/Reshape:0", shape=(?, 20), dtype=float32) Tensor("reshape_2/Reshape:0", shape=(?, 40), dtype=float32)
Tensor("concatenate_1/concat:0", shape=(?, 60), dtype=float32)


# y_second_order

In [12]:
latent = 8
'''Second Order Embeddings'''
emb_user_2 = Embedding(USER_DIM, latent)(user_inputs) # None * USER_DIM * K
emb_account_2 = Embedding(ACCOUNT_DIM, latent)(account_inputs) # None * ACCOUNT_DIM * K
print(emb_user_2, emb_account_2)

# emb_user_2 = RepeatVector(1)(MyMeanPool(axis=1)(emb_user_2))
# emb_account_2 = RepeatVector(1)(MyMeanPool(axis=1)(emb_account_2))
# print(emb_user_2, emb_account_2)

Tensor("embedding_3/embedding_lookup/Identity:0", shape=(?, 20, 8), dtype=float32) Tensor("embedding_4/embedding_lookup/Identity:0", shape=(?, 40, 8), dtype=float32)


In [13]:
emb = Concatenate(axis=1)([emb_user_2,emb_account_2]) # None * (USER_DIM+ ACCOUNT_DIM) * K
print(emb)

Tensor("concatenate_2/concat:0", shape=(?, 60, 8), dtype=float32)


In [16]:
'''compute FM part'''
summed_features_emb = MySumLayer(axis=1)(emb) # None * K
summed_features_emb_square = Multiply()([summed_features_emb,summed_features_emb]) # None * K

squared_features_emb = Multiply()([emb, emb]) # None * 9 * K
squared_sum_features_emb = MySumLayer(axis=1)(squared_features_emb) # Non * K
print(summed_features_emb_square, squared_sum_features_emb)

sub = Subtract()([summed_features_emb_square, squared_sum_features_emb]) # None * K
sub = Lambda(lambda x:x*0.5)(sub) # None * K
print(sub)
y_second_order = MySumLayer(axis=1)(sub) # None * 1

Tensor("multiply_1/mul:0", shape=(?, 8), dtype=float32) Tensor("my_sum_layer_2/Sum:0", shape=(?, 8), dtype=float32)
Tensor("lambda_1/mul:0", shape=(?, 8), dtype=float32)


In [17]:
y_second_order

<tf.Tensor 'my_sum_layer_3/Sum:0' shape=(?, 1) dtype=float32>

# deep parts

In [18]:
'''deep parts'''
y_deep = MyFlatten()(emb) # None*(6*K)
y_deep = Dropout(0.5)(Dense(128, activation='relu')(y_deep))
y_deep = Dropout(0.5)(Dense(64, activation='relu')(y_deep))
y_deep = Dropout(0.5)(Dense(32, activation='relu')(y_deep))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
y_deep

<tf.Tensor 'dropout_3/cond/Merge:0' shape=(?, 32) dtype=float32>

# concat - deepfm

In [20]:
'''deepFM'''
y = Concatenate(axis=1)([y_first_order, y_second_order, y_deep])
y = Dense(1, activation='sigmoid')(y)

model = Model(inputs=[user_inputs, account_inputs], outputs=[y])

In [21]:
plot_model(model, 'model.png', show_shapes=True)

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 20)           0                                            
__________________________________________________________________________________________________
account (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 20, 8)        160         user[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 40, 8)        320         account[0][0]                    
__________________________________________________________________________________________________
concatenat

In [29]:
adam = optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer = adam, loss = 'binary_crossentropy', sample_weight_mode = None) 

In [40]:
user_inputs1 = np.random.randint(0,2, (5120, USER_DIM))
account_inputs1 = np.random.randint(0,40, (5120, ACCOUNT_DIM))
label1 = np.random.randint(0,2, (5120, 1))

model.fit(
    [user_inputs_, account_inputs_],
    label_,
    epochs = 100,  # 300
    batch_size = 512,
    #verbose = 10,  ## 每个epoch输出一行记录
    #validation_split = 0.25,
    validation_data = ([user_inputs1, account_inputs1], label1),
    shuffle = True,
    class_weight = {0:1, 1:1}
)

Train on 5120 samples, validate on 5120 samples
Epoch 1/100
5120/5120 [==============================] - 1s 103us/step - loss: 0.0305 - val_loss: 3.8589
Epoch 2/100
5120/5120 [==============================] - 1s 110us/step - loss: 0.0225 - val_loss: 3.5935
Epoch 3/100
5120/5120 [==============================] - 1s 106us/step - loss: 0.0261 - val_loss: 3.6186
Epoch 4/100
5120/5120 [==============================] - 1s 106us/step - loss: 0.0276 - val_loss: 3.7094
Epoch 5/100
5120/5120 [==============================] - 1s 132us/step - loss: 0.0250 - val_loss: 3.5333
Epoch 6/100
5120/5120 [==============================] - 1s 112us/step - loss: 0.0305 - val_loss: 3.5763
Epoch 7/100
5120/5120 [==============================] - 1s 114us/step - loss: 0.0253 - val_loss: 3.7103
Epoch 8/100
5120/5120 [==============================] - 0s 94us/step - loss: 0.0304 - val_loss: 3.5706
Epoch 9/100
5120/5120 [==============================] - 1s 119us/step - loss: 0.0354 - val_loss: 3.5254
Epoch 10